In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [ ]:
# SGD를 이용해 행령 분해를 수행
# 원본 행렬 R
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
             [np.NaN, 5, np.NaN, 3, 1],
             [np.NaN, np.NaN, 3, 4, 4],
             [5, 2, 1, 2, np.NaN]])

num_users, num_items = R.shape

# 잠재 요인 차원 K = 3으로 설정
K = 3

# 분해 행렬 P와 Q 초기화 및 크기를 지정하고 정규 분포를 가진 임의의 값 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [ ]:
# 

In [ ]:
# 실제 R 행렬과 예측 행렬의 오차를 구하는 메소드
def get_rmse(R, P, Q, non_zeros ):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix  = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여
    # 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    return rmse

In [ ]:
# R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [ (i, j, R[i, j]) for i in range(num_users) 
             for j in range(num_items) if R[i, j] > 0 ]

# SGD를 반복해서 업데이트할 횟수
steps = 1000

# SGD 학습률
learning_rate = 0.01

# L2 규제 계수
r_lambda = 0.01

# SGD  기법으로 P와 Q 매트릭스를 계속 업데이트 
for step in range(steps) : 
    for i, j, r in non_zeros :
        # 실제 값과 예측 값의 차이인 오류 값 구하기
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # 규제를 반영한 SGD 업데이트 공식 적용
        P[i, :] = P[i, :] + learning_rate * (eij*Q[j, :] - r_lambda*P[i, :])
        Q[j, :] = Q[j, :] + learning_rate * (eij*P[i, :] - r_lambda*Q[j, :])
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 50 ) == 0 :
            print('### iteration step : ', step, ' rmse : ', rmse)

### iteration step :  0  rmse :  3.261355059488935
### iteration step :  0  rmse :  3.26040057174686
### iteration step :  0  rmse :  3.253984404542389
### iteration step :  0  rmse :  3.2521583839863624
### iteration step :  0  rmse :  3.252335303789125
### iteration step :  0  rmse :  3.251072196430487
### iteration step :  0  rmse :  3.2492449982564864
### iteration step :  0  rmse :  3.247416477570409
### iteration step :  0  rmse :  3.241926055455223
### iteration step :  0  rmse :  3.2400454107613084
### iteration step :  0  rmse :  3.240166740749792
### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.5003190892212748
### iteration step :  50  rmse :  0.5001616291326989
### iteration step :  50  rmse :  0.49899601202578087
### iteration step :  50  rmse :  0.4988483450145831
### iteration step :  50  rmse :  0.49895189256631756
### iteration step :  50  rmse :  0.49833236830090993
### iteration step :  50  rmse :  0.4984148489378701
### iterati

### iteration step :  700  rmse :  0.0166383624426085
### iteration step :  700  rmse :  0.016741936743323586
### iteration step :  700  rmse :  0.016603524189001625
### iteration step :  700  rmse :  0.016526454393300468
### iteration step :  700  rmse :  0.016639792083379498
### iteration step :  700  rmse :  0.016657201345297346
### iteration step :  700  rmse :  0.016472928381641428
### iteration step :  700  rmse :  0.01645241257047358
### iteration step :  700  rmse :  0.016138379086448083
### iteration step :  700  rmse :  0.016269993747904915
### iteration step :  700  rmse :  0.01635288508504558
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.01660906046895522
### iteration step :  750  rmse :  0.016708562969098305
### iteration step :  750  rmse :  0.016569153528341783
### iteration step :  750  rmse :  0.016493367054249922
### iteration step :  750  rmse :  0.016607027966870924
### iteration step :  750  rmse :  0.0166236810275254

In [ ]:
# 분해된 P와 Q 함수를 P * Q.T로 예측 행렬을 만들어서 출력하기
pred_matrix = np.dot(P, Q.T)
print('예측 행렬 : \n', np.round(pred_matrix, 3))

예측 행렬 : 
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


In [ ]:
# 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트
# 장르 속성을 이용한 영화 콘텐츠 기반 필터링 
# 콘텐츠 기반 필터링 : 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 구성 요소를 가진
# 다른 영화를 추천 ===> 영화 간의 유사성을 판단하는 기준이 영화를 구성하는 콘텐츠를 기반으로 하는 방식

# 따라서 실습으로 콘텐츠 기반 필터링 추천 시스템을 영화를 선택하는 데 중요한 요소인 영화 장르 속성을 기반을 수행해볼 예정
# ==> 장르 칼럼 값의 유사도를 비교한 뒤 그 중 높은 평점을 가지는 영화를 추천

In [ ]:
# 데이터 로딩
import pandas as pd 
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')


movies = pd.read_csv('./tmdb_5000_movies.csv')
# movies.head()
movies.info()
movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": ...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...",en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6194}, {""name"": ""DC E...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...",en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of Ameri

In [ ]:
# 데이터 가공
# 콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출하여 새로운 df 로 추출
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

# 복잡한 다중 데이터를 담고 있는 컬럼 풀어내기
# 1) 영화의 장르에는 여러 가지가 동시에 속할 수 있다. 하지만 컬럼 속 데이터 타입은 단순 문자열로 표기되어도 
#    안에 담겨 있는 값이 파이썬 리스트에 dict 형태로 담겨 있음에 유의하자
# 
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

# 여러 개의 개별 장르 데이터를 가지고 있고, 이 개별 장르의 명칭은 key인 name을 통해서 추출 가능

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [ ]:
# genres 컬럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출
from ast import literal_eval  # 파이썬 ast 모듈의 literal_eval 함수를 사용하여 이 문자열을 객체로 변환
print(type(movies_df['genres'][1]))
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
print(type(movies_df['genres'][1]))
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

<class 'str'>
<class 'list'>


In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [ ]:
# 장르 콘텐츠 유사도 측정
# 영화의 장르별 유사도를 측정하기 위해서 사용할 수 있는 방법에는 여러가지가 있을 수 있지만
# 가장 간단한 방법은 genres를 문자열로 변경한 뒤 이를 CountVecotrizer로 피처벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것임

# 단어 피처에 값을 부여할 때, 각 문서에서 해당 단어가 나타나는 횟수, 즉 Count를 부여하는 경우를 카운트 벡터화라고 함


# (1) 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환
# (2) genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교
#     이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성
# (3) 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천

###################################################################################################
# (1) genres 칼럼을 문자열로 변환한 뒤 사이킷런의 CountVectorizer를 이용해 피처 벡터 행렬로 만듬
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
# print(movies_df['genres_literal'][:1])

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat[0])
print(genre_mat.toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(count_vect.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.
# dir(movies_df['genres_literal'])

# genre_mat.toarray()

# (2) 생성된 피처 벡터 행렬에 사이킷런의 cosine_similarity()를 이용해 코사인 유사도 계산
# 사이킷런의 cosine_similarity() : 기준행과 비교행의 코사인 유사도를 행렬 형태로 반환하는 함수

# from sklearn.metrics.pairwise import cosine_similarity

# genre_sim = cosine_similarity(genre_mat, genre_mat)
# # 왼쪽 상단에서 오른쪽 하단으로 향하는 대각선은 자기 자신과의 유사도이므로 1
# # print(genre_sim.shape)
# # print(genre_sim[:1]) 

# # genre_sim 객체는 movies_df 데이터 프레임의 행별 장르 유사도 값을 가지고 있다. 
# # 이를 활용하여 movies_df 개별 레코드에 대해서 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야 함

# # (3) 기준 행별 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값을 추출
# genre_sim_sorted_ind = genre_si m.argsort()[:, ::-1] # 높은 순으로 정렬된 비교 행 위치의 인덱스 값을 가져옴
# print(genre_sim_sorted_ind[0]) 
# # 반환값 : [[   0 3494  813 ... 3038 3037 2401]]
# # 반환값의 의미 : 0번 레코드의 경우 자기 자신이 가장 유사도가 높고, 이를 제외하면 3494번 레코드가 가장 유사도가 높고
# # 그 다음은 813 레코드이며 가장 유사도가 낮은 레코드는 2041번이란 의미



  (0, 0)	1
  (0, 16)	1
  (0, 124)	1
  (0, 232)	1
  (0, 138)	1
  (0, 1)	1
  (0, 24)	1
  (0, 135)	1
  (0, 233)	1
[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'action': 0, 'adventure': 16, 'fantasy': 124, 'science': 232, 'fiction': 138, 'action adventure': 1, 'adventure fantasy': 24, 'fantasy science': 135, 'science fiction': 233, 'fantasy action': 125, 'crime': 64, 'adventure crime': 20, 'drama': 90, 'thriller': 234, 'action crime': 4, 'crime drama': 68, 'drama thriller': 106, 'adventure science': 29, 'animation': 33, 'family': 109, 'animation family': 38, 'fantasy family': 130, 'action science': 12, 'adventure action': 17, 'action thriller': 13, 'thriller crime': 238, 'western': 265, 'adventure western': 32, 'adventure family': 23, 'family fantasy': 115, 'fiction action': 139, 'action fantasy': 7, 'comedy': 44, 'action comedy': 3, 'comedy science': 59, 'adventure drama': 22, 'drama action': 91, 'romance': 214, '

In [ ]:
# 장르 콘텐츠 필터링을 이용한 영화 추천 함수 생성
# 인자 값 
# df : 기반 데이터
# sorted_ind : 레코드별 장르 코사인 유사도 인덱스를 가진 객체
# title_name : 고객이 선정한 추천 기준이 되는 영화 제목
# top_n : 추천할 영화 건수
# 반환 값 : 추천 영화 정보를 가진 df
def find_sim_moive(df, sorted_ind, title_name, top_n = 10) : 
    # 인자로 입력된 movies_df에서 'title' 칼럼이 입력된 title_name 값인 df 추출
    title_movie = df[df['title'] == title_name]
    
    # title_name을 가진 df의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index 출력. top_n index는 이차원 데이터
    # df에서 index로 활용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [ ]:
# 생성한 함수를 이용해 영화 대부와 장르별로 유사한 영화 10개 추천
similar_movies = find_sim_moive(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]
# movies_df[movies_df['title'] == 'The Godfather'] 3337

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [ ]:
# 추천 받은 영화들 중에 평점이 0.0인 영화 또는 추천하기에는 너무 낯선 영화 등이 존재
# 따라서 더 많은 후보군을 선정한 뒤에 영화의 평점에 따라 필터링해서 최종 추천하도록 변경

# 평화의 평균 평점 정보인 vote_average값을 적용 할 예정인데 투표자가 1, 2명인 경우에는 
# 왜곡 및 편향된 데이터를 가질 수 있음

# 0 ~ 10점까지 오름차순정렬하여 살펴봄
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [ ]:
# 따라서 이와 같은 왜곡된 평점 데이터를 회피할 수 있는 새로운 평가 방식을 적용
# 유명 영화 평점 사이트 IMDB에서 사용하는 평가 횟수에 대한 가중치가 부여된 평점 방식을 적용할 예정
# 가중 평점 공식 : (  V / (V + M)) * R + (M/(V+M)) * C
# V = 개별 영화에 평점을 투표한 횟수
# M = 평점을 부여하기 위한 최소 투표 횟수(전체 투포 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정함, 가중치 조절 역할 )
# R = 개별 영화에 대한 평균 평점
# C = 전체 영화에 대한 평균 평점


persentile = 0.6
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(persentile)
# print('C:', round(C, 3), 'm:', round(m, 3))

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m))* R)+((m/(m+v))*C)

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
movies_df[['title','vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [ ]:
# 새롭게 정의된 평점 기준에 따라서 영화를 추천
# 장르 유사성이 높은 영화를 top_n의 2배수 만큼 후보군으로 선정하고 가중치 평점이 높은 순으로 top_n만큼 추출하는 함수로 변경
def find_sim_moive(df, sorted_ind, title_name, top_n=10) :
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n)*2]
    similar_indexes = similar_indexes.reshape(-1)
    
    # 기준 영화 인덱스 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote_가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_moive(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


In [ ]:
# 아이템 기반 최근접 이웃 협업 필터링
# 최근접 이웃 협업 필터링은 사용자 기반과 아이템 기반으로 분류되는데 일반적으로 추천 정확도가 
# 더 뛰어난 아이템 기반의 협업 필터링을 구현해보겠음

# 협업 필터링 기반의 영화 추천을 위해서는 사용자가 영화의 평점을 매긴 사용자-영화 평점 행렬 데이터 세트가 필요
# MovieLens 데이터 세트 사용 : https://grouplens.org/datasets/movielens/latest/

In [ ]:
# 데이터 가공 및 변환
movies = pd.read_csv('./movies.csv') # 영화에 대한 메타 정보 : moive 아이디, 제목, 장르
ratings = pd.read_csv('./ratings.csv') # 사용자가 평점을 매긴 영화에 대한 정보 : 사용자id, moive id, 평점, 시간
print(moives.shape) 
moives.info()
print(ratings.shape)
ratings.info()

movies[:1]
# ratings[:1]

# ratings.describe()

(9742, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
(100836, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
# 협업 필터링은 ratings.csv와 같이 사용자와 아이템 간의 평점 (또는 다른 유형의 액션)에 기반한 추천시스템

# ratings df를 DataFrame의 pivot_table()함수로 로우레벨의 값을 칼럼으로 변경
print(ratings.loc[0])

# userid를 행으로하고 movieid를 컬럼으로 갖는 데이터 셋으로 변환
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
# 사용자가 평점을 매기지 않은 영화에 대한 평점은 NaN으로 할당됨

ratings_matrix.head(3)

userId               1.0
movieId              1.0
rating               4.0
timestamp    964982703.0
Name: 0, dtype: float64


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 또한 movieId로는 영화를 구분하기 어려우므로 제목으로 표현되도록 변경할 예정

# ratings와 moives를 조인하여 title 칼럼을 가져와 pivot_table() 인자로 사용할 것임

# title을 얻기 위해 movies와 조인
rating_movies = pd.merge(ratings, movies, on='movieId')

ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')
ratings_matrix.head(3)

# 기존 최소 평점이 0.5이므로 NaN 을 0으로 대체할 것임
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 변환된 사용자-영화 평점 행렬 데이터 세트를 사용하여 영화 간 유사도 측정
# 영화의 유사도는 코사인 유사도를 기반으로 측정함

# 사이킷런의 cosine_similarity()는 기준행과 타 행을 비교하여 유사도를 산출하는데
# 현재 데이터 셋에서 행은 사용자이므로 그대로 코사인 유사도를 측정하면 사용자간 유사도가 나오게 됨
# 따라서 영화를 기준으로 유사도를 평가하려면 ratings_matrix의 행 기준이 영화가 되고 열기준이 사용자가 되어야 함

# 판다스의 transpose()를 사용하여 전치 행렬 변경하여 새로운 행렬 생성
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 전치 행렬 형식으로 변경한 데이터 셋을 기반으로 영화의 코사인 유사도를 구해봄

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# 그리고 좀 더 직관적인 영화의 유사도 값을 표현하기 위해 코사인 유사도에 영화명을 매핑하여 df로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [ ]:
# 영화 대부와 유사도가 높은 영화 추출
item_sim_df['Godfather, The (1972)'].sort_values(ascending=False)[:6]

# 장르가 완전 다른 영화도 유사도가 매우 높게 나타남

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

In [ ]:
# 위에서 작성한 아이템 기반 유사도 데이터를 이용하여 개인에게 특화된 영화 추천 알고리즘 만들기

# 영화 유사도 데이터를 이용해 최근점 이웃 협업 필터링으로 개인에게 최적화된 영화 추천을 할 것인데
# 개인화된 영화 추천의 가장 큰 특징은 개인이 아직 보지 않은 영화를 추천한다는 것임
# 따라서 아직 관람하지 않은 영화에 대해서 아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기반으로 하여
# 새롭게 모든 영화의 예측 평점을 계산한 후 높은 예측 평점을 가진 영화를 추천하는 방식을 사용

# 먼저, N의 범위에 제약을 두지 않고 모든 아이템으로 가정하여 예측 평점을 구하는 로직 작성


# 영화 간 유사도를 가진 df와 사용자-영화 평점을 가진 df를 활용하여 사용자별 최적화된 평점 스코어를 예측하는 함수
# 인자 
# ratings_arr : 사용자-영화 평점 넘파이 행렬
# item_sim_arr : 영화 간 유사도 넘파이 행렬
# 반환값
# 개인화된 예측 평점
def predict_rating(ratings_arr, item_sim_arr):
    # N의 범위에 제약을 두지 않는다면 사용자별 영화 예측 평점은 사용자의 모든 영화에 대한 실제 평점과 
    # 영화의 다른 모든 영화와의 코사인 유사도를 벡터 내적 곱(dot)한 값을 정규화한 것을 의미
    ratings_pred = ratings_arr.dot(item_sim_arr)/np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred 

import numpy as np

ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


In [ ]:
# 예측 평점과 실제 평점과 차이를 확인해보기 위해 MSE 적용
# 다만 평점을 주지 않은 영화에도 예측 평점을 부여했기 때문에 실제와 예측 평점의 차이는 기존에 평점이 
# 부여된 데이터에 대해서만 오차 정도를 측정할 예정

from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함
def get_mse(pred, actual) :
    # 평점이 있는 실제 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 MSE : ', get_mse(ratings_pred, ratings_matrix.values))
# 실제 값과 예측 값의 스케일이 서로 다르기때문에 MSE가 클 수 있지만 중요한 것은 MSE를 감소시키는 방향으로 개선하는 것임


아이템 기반 모든 최근접 이웃 MSE :  9.895354759094706


In [ ]:
# 많은 영화의 유사도 벡터를 이용하다보니 상대적으로 평점 예측이 떨어졌는데
# 이를 특정 영화와 가장 비슷한 유사도를 가진 영화에 대해서만 유사도 벡터를 적용하도록 함수를 변경

# 다만 아래에 작성된 함수는 데이터의 크기가 커지면 매우 오래 걸리는 로직임
# 왜냐하면 행, 열별로 for 루프를 반복 수행하여 TOP_N 유사도 벡터를 계산하기 때문임

def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)
    
    # 사용자-아이템 평점 행렬의 열 크기만큼 루프 수행
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        
        # 개인화된 예측 평점 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
            
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
    return pred

ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print('아이템 기반 최근접 TOP-20 이웃 MSE:', get_mse(ratings_pred, ratings_matrix.values))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

아이템 기반 최근접 TOP-20 이웃 MSE: 3.695009387428144


In [ ]:
# 9번 userId 사용자가 어떤 영화를 좋아하는지 확인
# 사용자가 평점을 준 영화를 평점이 높은 순으로 나열
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[user_rating_id >0].sort_values(ascending=False)[:10]

# 전반적으로 흥행성이 좋은 영화에 높은 평점을 부여
# 위에서 만든 아이템 기반 협업 필터링을 통해 영화 추천

# 사용자가 이미 평점을 준 영화를 제외하고 추천할 수 있도록 평점을 주지 않은 영화를 리스트 객체로 반환
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 정보를 추출하여 Series로 반환함
    # 반환된 user_rating은 영화명을 인덱스로 가지는 Series 객체임
    user_rating = ratings_matrix.loc[userId , :]
    
    # user_rating이 0보다 크면 기존에 관람한 영화이므로 대상 인덱스를 추출하여 list 객체를 만듬
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듦.
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 영화는 moives_list에서 제외함
    unseen_list = [x for x in movies_list if x not in already_seen]
    
    return unseen_list

# 사용자가 영화의 평점을 주지 않은 추천 대상 영화 정보와 사용자별 아이템 유사도에 기반한 예측 평점 데이터 세트를
# 활용하여 사용자에게 영화를 추천하는 함수
# 인자 
# 예측 평점 df, 추천하려는 사용쟈id, 추천 후보 영화 리스트, 추천 상위 영화 개수
# 반환값
# 사용자가 좋아할만한 가장 높은 예측 평점을 가진 영화

def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 df에서 사용자id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출하여 가장 예측 평점이 높은 순으로 정렬함
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 최근접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 df으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


In [ ]:
# 행렬 분해를 이용한 잠재 요인 협업 필터링 실습
# SVD, NMF 등 적용 가능
# 일반적으로 행렬 분해에는 SVD가 자주 사용되지만 사용자-아이템 평점 행렬에는
# 사용자가 평점을 매기지 않은 널 데이터가 많기 때문에 주로 SGD나 ALS 기반 행렬 분해를 이용

# SGD 기반의 행렬 분해를 구현하고 이를 기반으로 사용자에게 영화를 추천할 것임

# 확률적 경사 하강법을 이용한 행렬 분해 예지의 get_rmse()함수를 그대로 사용하되
# 해렬 분해 로직을 새로운 matrix_factorization()로 정리

# 실제 R 행렬과 예측 행렬의 오차를 구하는 메소드
def get_rmse(R, P, Q, non_zeros ):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix  = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여
    # 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    return rmse

def matrix_factorization(R, K, steps=200,learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size = (num_users, K))
    Q = np.random.normal(scale=1./K, size= (num_items, K))
    
    prev_rmse = 10000
    break_count = 0
    
    # R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
    non_zeros = [ (i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] >0]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i, :] = P[i, :] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
            Q[j, :] = Q[j, :] + learning_rate*(eij * P[i, :] - r_lambda*Q[j, :])
            
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print('### interation step : ', step, ' rmse :', rmse)
    return P, Q



In [ ]:
# 영화 평점 행렬 데이터를 새롭게 df로 로딩한 뒤 다시 사용자-아이템 평점 행렬로 만들기
import pandas as pd
import numpy as np

movies = pd.read_csv('./movies.csv') # 영화에 대한 메타 정보 : moive 아이디, 제목, 장르
ratings = pd.read_csv('./ratings.csv') # 사용자가 평점을 매긴 영화에 대한 정보 : 사용자id, moive id, 평점, 시간

ratings_movies = pd.merge(ratings, movies, on='movieId')
ratings_movies = ratings_movies[['rating', 'userId', 'movieId', 'title']]

ratings_matrix = ratings_movies.pivot_table('rating', index = 'userId', columns='title')

# 행렬 분해 수행
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix=np.dot(P, Q.T)

ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

#  평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

### interation step :  0  rmse : 2.9023619751336867
### interation step :  10  rmse : 0.7335768591017927
### interation step :  20  rmse : 0.5115539026853442
### interation step :  30  rmse : 0.37261628282537446
### interation step :  40  rmse : 0.2960818299181014
### interation step :  50  rmse : 0.2520353192341642
### interation step :  60  rmse : 0.22487503275269854
### interation step :  70  rmse : 0.20685455302331537
### interation step :  80  rmse : 0.19413418783028685
### interation step :  90  rmse : 0.18470082002720403
### interation step :  100  rmse : 0.17742927527209104
### interation step :  110  rmse : 0.17165226964707486
### interation step :  120  rmse : 0.1669518194687172
### interation step :  130  rmse : 0.16305292191997542
### interation step :  140  rmse : 0.15976691929679643
### interation step :  150  rmse : 0.1569598699945732
### interation step :  160  rmse : 0.15453398186715428
### interation step :  170  rmse : 0.15241618551077643
### interation step :  180  

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690
